# PyCity Schools Analysis

- The most obvious observed trend was that Reading scores were superior to math across all schools. Reading passing rate was 100%, compared to math's around 89%.

- Charter school results had higher average scores in both math and reading. Also ,the smaller the school, the more likely it was to have higher reading and math scores.

- Counter-intuitively, *lower* spending ranges per student resulted in higher average math and reading scores. 


In [1]:
# Dependencies
import pandas as pd

In [6]:
# load CSVs
schools_csv  = "raw_data/schools_complete.csv"
students_csv = "raw_data/students_complete.csv"

## Overall District Summary
#### Read each CSV file with pandas, determine average scores/passing rate, and aggregate into summary dataframe

In [8]:
dfSchools = pd.read_csv(schools_csv, encoding="iso-8859-1", low_memory=False)
dfStudents = pd.read_csv(students_csv, encoding="iso-8859-1", low_memory=False)

# *** Determine total number of schools in district ***
dfDistrictSchools = dfSchools.loc[dfSchools["type"] == "District"]

# Determine school count
num_schools_in_district = len(dfDistrictSchools)

# *** Determine total number of students in districts ***
# Rename ambiguous column names
dfSchools = dfSchools.rename(columns={"name": "school_name"})
dfStudents = dfStudents.rename(columns={"school": "school_name", "name": "student_name"})

# Merge schools and students dataframes using an inner join
dfStudentsInSchools = pd.merge(dfSchools, dfStudents, on="school_name")

# Filter merged dataframe by just district school students
num_students_in_districts = dfStudentsInSchools.loc[dfStudentsInSchools["type"] == "District"].count()["student_name"]

# Determine / format total budget for district
district_budget = dfDistrictSchools["budget"].sum()
district_budget = "${:,.2f}".format(district_budget)

# Determine average math score
average_math_score = dfStudentsInDistricts["math_score"].mean()
average_math_score = round(average_math_score, 6)

# Determine average reading score
average_reading_score = dfStudentsInDistricts["reading_score"].mean()
average_reading_score = round(average_reading_score, 6)

# Determine % passing math
num_students_passing_math = dfStudentsInDistricts.loc[dfStudentsInDistricts["math_score"] > 59].count()["student_name"]
percent_students_passing_math = (num_students_passing_math / num_students_in_districts) * 100

# Determine % passing reading
num_students_passing_reading = dfStudentsInDistricts.loc[dfStudentsInDistricts["reading_score"] > 59].count()["student_name"]
percent_students_passing_reading = (num_students_passing_reading / num_students_in_districts) * 100

# Determine overall passing rate (average of math/reading)
overall_passing_rate = round(((percent_students_passing_math + percent_students_passing_reading) / 2), 5)

# Set up summary table
summary_table_data = {'Total Schools': [num_schools_in_district],
                      'Total Students': [num_students_in_district],
                      'Total Budget': [district_budget], 
                      'Average Math Score': [average_math_score],
                      'Average Reading Score': [average_reading_score],
                      '% Passing Math': [percent_students_passing_math],
                      '% Passing Reading': [percent_students_passing_reading],
                      '% Overall Passing Rate': [overall_passing_rate]
                     }
dfSummary = pd.DataFrame(summary_table_data)
dfSummary = dfSummary[
                        ["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score",
                          "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
                     ]
dfSummary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,7,26976,"$17,347,923.00",76.987026,80.962485,89.030991,100.0,94.5155


## Performance Summary by School
#### Build function to return a Dataframe summarizing the schools data

In [43]:
''' 
Function:   schools_summary
Argument:   Merged "students by schools" dataframe
Returns:    Schools performance summary dataFrame
'''
def schools_summary(df_students_in_schools):
    # Create groupby object for schools
    schools_grouping = df_students_in_schools.groupby("School ID")

    # Create a master dataframe to hold schools totals 
    dfSchoolTotals = pd.DataFrame(schools_grouping["school_name"].size()).reset_index()
    dfSchoolTotals = dfSchoolTotals.rename(columns={"school_name": "number_of_students"})

    # Find out the average math and reading scores, and build dataframes for them 
    dfSchools_MathAverage = pd.DataFrame(schools_grouping["math_score"].mean()).reset_index()
    dfSchools_ReadingAverage = pd.DataFrame(schools_grouping["reading_score"].mean()).reset_index()

    # Rename the column to be more specific
    dfSchools_MathAverage = dfSchools_MathAverage.rename(columns={"math_score": "average_math_score"})
    dfSchools_ReadingAverage = dfSchools_ReadingAverage.rename(columns={"reading_score": "average_reading_score"})

    # Merge the dataframes with dfSchoolTotals to form new summary dataframe
    dfSchoolsSummary = pd.merge(dfSchoolTotals, dfSchools_MathAverage, on="School ID")
    dfSchoolsSummary = pd.merge(dfSchoolsSummary, dfSchools_ReadingAverage, on="School ID")

    # Determine the % Passing math
    dfStudentsPassingMath = df_students_in_schools[df_students_in_schools["math_score"] > 59]
    schools_grouping_passingmath = dfStudentsPassingMath.groupby("School ID")
    dfSchools_PassingMath = schools_grouping_passingmath["student_name"].count().reset_index()
    dfSchools_PassingMath = dfSchools_PassingMath.rename(columns={"student_name": "num_students_passedmath"})

    # Merge num_students_passed_math column into our summary dataframe
    dfSchoolsSummary = pd.merge(dfSchoolsSummary, dfSchools_PassingMath, on="School ID")

    # Determine the % Passing reading
    dfStudentsPassingReading = df_students_in_schools[df_students_in_schools["reading_score"] > 59]
    schools_grouping_passingreading = dfStudentsPassingReading.groupby("School ID")
    dfSchools_PassingReading = schools_grouping_passingreading["student_name"].count().reset_index()
    dfSchools_PassingReading = dfSchools_PassingReading.rename(columns={"student_name": "num_students_passedreading"})

    # Merge num_students_passed_reading column back into our summary dataframe
    dfSchoolsSummary = pd.merge(dfSchoolsSummary, dfSchools_PassingReading, on="School ID")

    # Add a column for the percentage of students passing math & reading, based on the total number of students at the school
    dfSchoolsSummary["percentage_students_passedmath"] = (dfSchoolsSummary["num_students_passedmath"]/dfSchoolsSummary["number_of_students"])*100
    dfSchoolsSummary["percentage_students_passedreading"] = (dfSchoolsSummary["num_students_passedreading"]/dfSchoolsSummary["number_of_students"])*100

    # Add a column for the % Overall Passing Rate
    dfSchoolsSummary["overall_passing_rate"] = (dfSchoolsSummary["percentage_students_passedmath"] + dfSchoolsSummary["percentage_students_passedreading"])/2

    return dfSchoolsSummary


#### Create school summary dataframe, and perform additional customizations on it. Return dataframe to visualize data.

In [45]:
# Put performance summary dataframe into variable
dfSchoolsSummary = schools_summary(dfStudentsInSchools)

# Merge our summary table back with the main Schools DF to append the remaining needed columns
dfSchoolSummaryTable = pd.merge(dfSchoolsSummary, dfSchools, on="School ID")

# Add the 'Per Student Budget' column
dfSchoolSummaryTable["Per Student Budget"] = dfSchoolSummaryTable["budget"] / dfSchoolSummaryTable["number_of_students"]
    
# Drop the columns we no longer need for display
dfSchoolSummaryTable.drop(["School ID", "num_students_passedreading", "num_students_passedmath", "size"], axis=1, inplace=True)

# Format the budget columns as currency
dfSchoolSummaryTable["budget"] = dfSchoolSummaryTable["budget"].map("${:,.2f}".format)
dfSchoolSummaryTable["Per Student Budget"] = dfSchoolSummaryTable["Per Student Budget"].map("${:,.2f}".format)

# Rename the columns to be more human-readable
dfSchoolSummaryTable = dfSchoolSummaryTable.rename(columns={"number_of_students": "Total Students", 
                                                    "average_math_score": "Average Math Score",
                                                   "average_reading_score": "Average Reading Score",
                                                    "percentage_students_passedmath": "% Passing Math",
                                                    "percentage_students_passedreading": "% Passing Reading",
                                                    "overall_passing_rate": "% Overall Passing Rate",
                                                    "type": "School Type",
                                                    "budget": "Total School Budget",
                                                    "school_name": "Name"
                                                       }
                                                    )
# Order the columns, set index to school "name"
dfSchoolSummaryTable = dfSchoolSummaryTable[['Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 
                                             'Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', 
                                             '% Overall Passing Rate'
                                            ]].set_index("Name")
dfSchoolSummaryTable.head(30)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,88.858416,100.0,94.429208
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,88.436758,100.0,94.218379
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,100.000000,100.0,100.000000
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,89.083064,100.0,94.541532
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,100.000000,100.0,100.000000
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,100.000000,100.0,100.000000
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,100.000000,100.0,100.000000
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,89.529743,100.0,94.764871
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,100.000000,100.0,100.000000


### Summary of top 5 performing schools

In [46]:
# Sort the Schools dataframe by the Overall Rating
dfSchoolSummaryTable = dfSchoolSummaryTable.sort_values("% Overall Passing Rate", ascending=False)

# Output the top 5 performing schools
dfSchoolSummaryTable.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Name,,,,,,,,,
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,100.0,100.0,100.0
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,100.0,100.0,100.0
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,100.0,100.0,100.0
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,100.0,100.0,100.0
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,100.0,100.0,100.0


### Summary of bottom 5 performing schools

In [47]:
# Sort the Schools dataframe by the Overall Rating
dfSchoolSummaryTable = dfSchoolSummaryTable.sort_values("% Overall Passing Rate", ascending=True)

# Output the bottom 5 performing schools
dfSchoolSummaryTable.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Name,,,,,,,,,
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,88.436758,100.0,94.218379
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,88.547137,100.0,94.273568
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,88.858416,100.0,94.429208
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,89.083064,100.0,94.541532
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,89.182945,100.0,94.591472


## Performance by Grade Level
#### Create a table that lists the average Math/Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [35]:
'''
Function:   average_scores_by_grade
Argument:   score type ("math" or "reading")
Returns:
            Dataframe of math/reading score averages
'''
def average_scores_by_grade(type):
    # Define column we are filtering by (e.g., "math_score")
    col = f"{type}_score"
    
    # Find out the average math and reading scores for each grade on the schools grouping, 
    #  and build dataframes for them:
    #
    # For students in each grade, group by school, and aggregate average score (math or reading)
    dfGrade9_Average = dfStudentsInSchools.loc[dfStudentsInSchools["grade"] == "9th"].groupby("school_name")[col].mean().reset_index()
    dfGrade10_Average = dfStudentsInSchools.loc[dfStudentsInSchools["grade"] == "10th"].groupby("school_name")[col].mean().reset_index()
    dfGrade11_Average = dfStudentsInSchools.loc[dfStudentsInSchools["grade"] == "11th"].groupby("school_name")[col].mean().reset_index()
    dfGrade12_Average = dfStudentsInSchools.loc[dfStudentsInSchools["grade"] == "12th"].groupby("school_name")[col].mean().reset_index()
    
    # Repurpose our Schools summary table again as a new summary dataframe
    dfAverages = dfSchoolSummaryTable.reset_index()

    # Merge 9th and 10th grade-specific math/reading averages with the new summary dataframe
    dfAverages = pd.merge(dfAverages, dfGrade9_Average, left_on=['Name'], right_on=['school_name'])
    dfAverages = pd.merge(dfAverages, dfGrade10_Average, left_on=['Name'], right_on=['school_name'])
    dfAverages = dfAverages.rename(columns={f"{col}_x": "9th", f"{col}_y": "10th"})  # Rename columns
    dfAverages = pd.merge(dfAverages, dfGrade11_Average, left_on=['Name'], right_on=['school_name'])
    dfAverages = pd.merge(dfAverages, dfGrade12_Average, left_on=['Name'], right_on=['school_name'])
    dfAverages = dfAverages.rename(columns={f"{col}_x": "11th", f"{col}_y": "12th"})  # Rename columns
    
    # Remove the extra Schools summary columns we don't need, then the extra school name columns
    dfAverages.drop(dfAverages.columns[1:10], axis=1, inplace=True)
    dfAverages.drop(dfAverages.columns[[1,3,5,7]], axis=1, inplace=True)

    # Reset index to school name, and rename columns
    dfAverages = dfAverages.set_index("Name")
    
    # Return cleaned up dataframe
    return dfAverages


### Average Math Scores by Grade

In [36]:
# Output summary table for average math scores by grade 
dfMathSummary = average_scores_by_grade("math")

dfMathSummary.head(30)

,9th,10th,11th,12th
Name,,,,
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Rodriguez High School,76.859966,76.612500,76.395626,77.690748
Huang High School,77.027251,75.908735,76.446602,77.225641
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Johnson High School,77.187857,76.691117,77.491653,76.863248
Ford High School,77.361345,77.672316,76.918058,76.179963
Bailey High School,77.083676,76.996772,77.515588,76.492218
Shelton High School,83.420755,82.917411,83.383495,83.778976
Griffin High School,82.044010,84.229064,83.842105,83.356164


### Average Reading Scores by Grade

In [48]:
# Output summary table for average reading scores by grade 
dfReadingSummary = average_scores_by_grade("reading")

dfReadingSummary.head(30)

,9th,10th,11th,12th
Name,,,,
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Rodriguez High School,80.993127,80.629808,80.864811,80.376426
Huang High School,81.290284,81.512386,81.417476,80.305983
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Johnson High School,81.260714,80.773431,80.616027,81.227564
Ford High School,80.632653,81.262712,80.403642,80.662338
Bailey High School,81.303155,80.907183,80.945643,80.912451
Shelton High School,84.122642,83.441964,84.373786,82.781671
Griffin High School,83.369193,83.706897,84.288089,84.013699


## School Performance by Size/Type/Budget

### Create school averages grouping functions
#### Function outputting a table grouping the average math/reading scores and % Passing Math/Reading/Overall rates, given a specific grouping column

In [38]:
'''
Function:    school_averages_by_group
Arguments:
             Argument 1:  Summary dataframe
             Argument 2:  Column to group by
Returns:
            Summary table dataframe
'''
def school_averages_by_group(df, col):
    # Create a group based off of the bins
    grouping = df.groupby(col)

    # Create dataframes to house our math / reading averages for the groups
    dfMathAveragesByGroups = pd.DataFrame(grouping["Average Math Score"].mean()).reset_index()
    #dfMathAveragesByGroups.head()
    dfReadingAveragesByGroups = pd.DataFrame(grouping["Average Reading Score"].mean()).reset_index()

    # Create dataframes to house our % passing math/reading, overall passing rate columns for the groups
    dfPercentPassingMathByGroups = pd.DataFrame(grouping["% Passing Math"].mean()).reset_index()
    dfPercentPassingReadingByGroups = pd.DataFrame(grouping["% Passing Reading"].mean()).reset_index()
    dfPercentOverallPassingByGroups = pd.DataFrame(grouping["% Overall Passing Rate"].mean()).reset_index()

    # Merge all the dataframes into summary frame
    dfSummaryTable = pd.merge(dfMathAveragesByGroups, dfReadingAveragesByGroups, on=col)
    dfSummaryTable = pd.merge(dfSummaryTable, dfPercentPassingMathByGroups, on=col)
    dfSummaryTable = pd.merge(dfSummaryTable, dfPercentPassingReadingByGroups, on=col)
    dfSummaryTable = pd.merge(dfSummaryTable, dfPercentOverallPassingByGroups, on=col)

    # Set the index to the binning column 
    dfSummaryTable = dfSummaryTable.set_index(col)
    
    return dfSummaryTable

#### Function outputting a table that breaks down school performances based on specified binning & grouping columns

In [49]:
'''
Function:   schools_bin_summary
Arguments:
            Argument 1:  Summary dataframe
            Argument 2:  Column to create
            Argument 3:  Data Column to bin
            Argument 4:  Bins (numeric)
            Argument 5:  Bin groups (string) 
Returns:
            Summary table dataframe
'''
def schools_bin_summary(df, col_to_bin, col_to_group, bins, groups):
    # Convert the column to group back to numeric so that we can "bin" it (take any "$" characters out)
    df[col_to_group] = df[col_to_group].replace('[\$,)]','', regex=True).astype(float)

    # Cut col_to_group column (e.g.,'Per Student Budget') values into bins, and add new 
    #  column to dataframe for our new binned values
    df[col_to_bin] = pd.cut(df[col_to_group], bins, labels=groups)

    # Call function to give us a grouped dataframe of all the averages
    dfSummaryTable = school_averages_by_group(df, col_to_bin)

    # Reorder the rows according to their order in the "groups" list
    dfSummaryTable = dfSummaryTable.loc[groups,]

    # Return the binned dataframe
    return dfSummaryTable

### School Performance by Budget

In [40]:
# Create the bins in which Data will be held
bins_list = [0, 585, 615, 645, 675]

# Create the names for the four bins
group_names = ['<$585', '$585-615', '$615-645', '$645-675']

# Call the function to output the dataframe for this binned table
dfSpendingRanges = schools_bin_summary(
                                        dfSchoolSummaryTable, 
                                        "Spending Ranges (Per Student)", 
                                        "Per Student Budget", 
                                        bins_list, 
                                        group_names
                                      )
dfSpendingRanges.head(10)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,100.000000,100.0,100.000000
$585-615,83.599686,83.885211,100.000000,100.0,100.000000
$615-645,79.079225,81.891436,92.636050,100.0,96.318025
$645-675,76.997210,81.027843,89.041475,100.0,94.520737


### School Performance by Size

In [41]:
# Create the bins in which Data will be held
bins_list = [0, 1000, 2000, 3000, 5000]

# Create the names for the bins
group_names = ['Small (<1000)', 'Medium-Small (1000-2000)', 'Medium-Large (2000-3000)', 'Large (3000-5000)']

# Call the function with our summary table, column names, and bins info to output the dataframe for this binned table
dfSchoolSizeSummary = schools_bin_summary(
                                        dfSchoolSummaryTable, 
                                        "School Size", 
                                        "Total Students", 
                                        bins_list, 
                                        group_names
                                      )
# Reorder the results in reverse alpha order so we go from Small to Large
dfSchoolSizeSummary.head(10)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,100.000000,100.0,100.000000
Medium-Small (1000-2000),83.374684,83.864438,100.000000,100.0,100.000000
Medium-Large (2000-3000),78.429493,81.769122,91.649460,100.0,95.824730
Large (3000-5000),77.063340,80.919864,89.085722,100.0,94.542861


### School Performance by Type

In [50]:
# Set our column to group
col_to_group = "School Type"

# Pass summary table dataframe and grouping column to our utility function to obtain dataframe of grouped averages
dfSchoolTypeSummary = school_averages_by_group (dfSchoolSummaryTable, col_to_group)

# Output dataframe
dfSchoolTypeSummary.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,100.000000,100.0,100.000000
District,76.956733,80.966636,88.991533,100.0,94.495766
